In [30]:
using GamsStructure
using CSV

In [31]:
set_dir = "base_data_2017/set"
parm_dir = "base_data_2017/parameter"

GU = GamsUniverse()


@GamsSets(GU,set_dir,begin
    :sr, "Super Regions in WiNDC Database"
    :r, "Regions in WiNDC Database"
    #:yr, "Years in WiNDC Database"
    :i, "BEA Goods and sectors categories"
end)

alias(GU,:i,:s)

@GamsDomainSets(GU,parm_dir,begin
    :n, :usatrd_units, 2,"Dynamically created set from parameter usatrd, NAICS codes"
    :yr, :usatrd_units,   3, "Dynamically created set from parameter usatrd_units, Years in USA trade data set (2002-2016)"
    :t, :usatrd_units, 4, "Dynamically create set from parameter usatrd, Trade type (import/export)"
    :usa_units_domain, :usatrd_units, 5, "Dynamically create set from parameter usatrd, Units"
end)

@GamsParameters(GU,parm_dir,begin
    :usatrd_units, (:sr,:n,:yr,:t,:usa_units_domain), "Trade data with units as domain", [1,2,3,4,5]
end)

@GamsParameters(GU,begin
    :usatrd,    (:yr,:r,:i,:t), "Mapped trade data" #"Trade data without units"
    #:usatrd_,   (:yr,:r,:i,:t), "Mapped trade data"
    :usatrd_shr,(:yr,:r,:i,:t), "Share of total trade by region"
    :shrchk,    (:s,:t), "Comparison between imports and exports"
end)

1;

In [32]:
function load_usatrd_map(;map_file ="maps/mapusatrd.csv" )

    mappce = Dict()
    F = CSV.File(map_file,stringtype=String,
                silencewarnings=true,header = [:a,:b,:c])

    for row in F
        val = Symbol(row[1])
        key = Symbol(string(row[2]))
        if val ∈ keys(mappce)
            push!(mappce[val],key)
        else
            mappce[val] = [key]
        end
    end
    return mappce
end

usatrd_map = load_usatrd_map();

In [48]:
"Scale originally millions. Make billions"

for i∈keys(usatrd_map)
    if length(usatrd_map[i])==1
        GU[:usatrd][:yr,:r,[i],:t] = permutedims(GU[:usatrd_units][:r,usatrd_map[i],:yr,:t,:],(2,1,3))*1e-3
    else
        GU[:usatrd][:yr,:r,[i],:t] = permutedims(sum(GU[:usatrd_units][:r,usatrd_map[i],:yr,:t,:],dims=2),(3,1,2,4))*1e-3
    end
end

In [54]:
notinc = [s for s∈GU[:i] if s∉keys(usatrd_map)]
inc = [s for s∈GU[:i] if s∈keys(usatrd_map)]

#for r∈GU[:r]
#    mask = sum(GU[:usatrd][:yr,:r,inc,:t],dims=2)!=0


#GU[:usatrd_shr][:yr,:r,:s,:t]

1;

In [50]:
usatrd_shr(yr,r,s,t)$(NOT notinc(s) and sum(r.local, usatrd_(yr,r,s,t))) = usatrd_(yr,r,s,t) / sum(r.local, usatrd_(yr,r,s,t));


In [56]:
mask = sum(GU[:usatrd][:yr,[:IA],inc,:t],dims=2).!=0

16×1×2 BitArray{3}:
[:, :, 1] =
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

[:, :, 2] =
 1
 1
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 0
 1
 0
 1